In [1]:
import duckdb
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from pathlib import Path 
# df = pd.read_parquet('datasets/competencia_01_aum.parquet')


# tabla = pa.Table.from_pandas(df)

# # write df as a parquet dataset
# ds.write_dataset(tabla, base_dir = "datasets/competencia_01_aumpy",
#                   format="parquet",
#                  partitioning= ["foto_mes"],
#                  max_rows_per_file = 42000, max_rows_per_group = 42000,
#                  existing_data_behavior = "overwrite_or_ignore")

ModuleNotFoundError: No module named 'pyarrow'

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour
from time import time
import pickle
import os
from dotenv import load_dotenv
import urllib


In [3]:

def preparar_data(dbname, mes_train, mes_test, drop_cols=None, sampling = 1):

        #%%
    import pandas as pd
    import numpy as np
    import seaborn as sns
    import matplotlib.pyplot as plt
    from sklearn.model_selection import train_test_split
    from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
    from sklearn.ensemble import RandomForestClassifier
    import lightgbm as lgb
    import optuna
    from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour
    from time import time
    import pickle
    import os
    from dotenv import load_dotenv
    import urllib
    import pyarrow.dataset as ds
    import lgbm_globales
    import duckdb
    from pathlib import Path


    print('Preparando data...')

    root_path = Path(__file__).parent.parent.resolve()

    root_path = root_path.as_posix() + '/'

    # Connect to DuckDB database
    con = duckdb.connect(root_path+'duckdb/'+ dbname + ".duckdb")       
    
    query = (f"""
            SELECT *
            FROM {dbname}
            JOIN {dbname}_lags
            ON {dbname}.foto_mes = {dbname}_lags.foto_mes
            AND {dbname}.numero_de_cliente = {dbname}_lags.numero_de_cliente
            AND {dbname}.clase_ternaria = {dbname}_lags.clase_ternaria;""")

    # Execute the join query and fetch the result as a Pandas DataFrame
    data = con.execute(query).fetchdf()

    # data = pd.read_parquet(dataset_path + dataset_file)

    data['clase_peso'] = 1.0

    data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
    data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

    data['clase_binaria1'] = 0
    data['clase_binaria2'] = 0
    data['clase_binaria1'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)
    data['clase_binaria2'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1)

    data = data.loc[data['foto_mes'].isin(mes_train + mes_test)]

    data_continua = data[data['clase_ternaria'] == 'CONTINUA']

    data_continua = data_continua.groupby('foto_mes').sample(frac=sampling, random_state=42)

    data = pd.concat[data_continua, data[data['clase_ternaria'] != 'CONTINUA']]

    print(data.head())

    del data_continua

    train_data = data.loc[data['foto_mes'].isin(mes_train)]
    test_data = data[data['foto_mes'] == mes_test]

    X_train = train_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
    y_train_binaria1 = train_data['clase_binaria1']
    y_train_binaria2 = train_data['clase_binaria2']
    w_train = train_data['clase_peso']

    X_test = test_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
    y_test_binaria1 = test_data['clase_binaria1']
    y_test_class = test_data['clase_ternaria']
    w_test = test_data['clase_peso']

    print('Data preparada')

    return X_train, y_train_binaria1, y_train_binaria2, w_train, X_test, y_test_class, y_test_binaria1, w_test



In [3]:

dbname = "compe_02"

In [4]:


# Connect to DuckDB database
con = duckdb.connect('duckdb/'+ dbname + ".duckdb")

In [8]:
query = (f"""
        SELECT *
        FROM {dbname}
        JOIN {dbname}_lags
        ON {dbname}.foto_mes = {dbname}_lags.foto_mes
        AND {dbname}.numero_de_cliente = {dbname}_lags.numero_de_cliente
        AND {dbname}.clase_ternaria = {dbname}_lags.clase_ternaria
        LIMIT 5000;""")

# query = (f"""
#         SELECT *
#         FROM {dbname}_lags
#         LIMIT 5000;""")

# Execute the join query and fetch the result as a Pandas DataFrame
data = con.execute(query).fetchdf()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [9]:
data.shape

(5000, 702)

In [11]:
data.columns.tolist()

['numero_de_cliente',
 'foto_mes',
 'active_quarter',
 'cliente_vip',
 'internet',
 'cliente_edad',
 'cliente_antiguedad',
 'mrentabilidad',
 'mrentabilidad_annual',
 'mcomisiones',
 'mactivos_margen',
 'mpasivos_margen',
 'cproductos',
 'tcuentas',
 'ccuenta_corriente',
 'mcuenta_corriente_adicional',
 'mcuenta_corriente',
 'ccaja_ahorro',
 'mcaja_ahorro',
 'mcaja_ahorro_adicional',
 'mcaja_ahorro_dolares',
 'cdescubierto_preacordado',
 'mcuentas_saldo',
 'ctarjeta_debito',
 'ctarjeta_debito_transacciones',
 'mautoservicio',
 'ctarjeta_visa',
 'ctarjeta_visa_transacciones',
 'mtarjeta_visa_consumo',
 'ctarjeta_master',
 'ctarjeta_master_transacciones',
 'mtarjeta_master_consumo',
 'cprestamos_personales',
 'mprestamos_personales',
 'cprestamos_prendarios',
 'mprestamos_prendarios',
 'cprestamos_hipotecarios',
 'mprestamos_hipotecarios',
 'cplazo_fijo',
 'mplazo_fijo_dolares',
 'mplazo_fijo_pesos',
 'cinversion1',
 'minversion1_pesos',
 'minversion1_dolares',
 'cinversion2',
 'minversi

In [26]:
query = (f"""
        SELECT *
        FROM {dbname}
        JOIN {dbname}_lags
        ON {dbname}.foto_mes = {dbname}_lags.foto_mes
        AND {dbname}.numero_de_cliente = {dbname}_lags.numero_de_cliente
        AND {dbname}.clase_ternaria = {dbname}_lags.clase_ternaria
        LIMIT 500000;""")

# Execute the join query and fetch the result as a Pandas DataFrame
data = con.execute(query).fetchdf()

data

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,tc_proporcion_consumo_pesos,tc_proporcion_consumo_dolares,tc_proporcion_consumo_total_limite_total_cubierto,tc_proporcion_pago_pesos,tc_proporcion_pago_dolares,tc_proporcion_adelanto_pesos,tc_proporcion_adelanto_dolares,numero_de_cliente_1,foto_mes_1,clase_ternaria_1
0,249221109,201901,1,0,1,59,276,7597.55,47433.58,5654.59,...,1.0,0.000129,0.061601,-6.188851,0.000366,NaN,NaN,249221109,201901,CONTINUA
1,249221109,201902,1,0,1,59,277,3800.93,46605.81,2104.71,...,1.0,0.000262,0.051877,-4.021923,0.000065,NaN,NaN,249221109,201902,CONTINUA
2,249221109,201903,1,0,1,59,278,4102.67,45348.80,2492.53,...,1.0,0.000176,0.061586,-2.142776,0.001116,NaN,NaN,249221109,201903,CONTINUA
3,249221109,201904,1,0,1,60,279,4374.99,44872.11,3231.46,...,1.0,0.000053,0.170511,-4.470767,0.000000,NaN,NaN,249221109,201904,CONTINUA
4,249221109,201907,1,0,1,60,282,3341.93,44801.06,2727.38,...,1.0,0.000122,0.076126,-4.202987,0.000000,NaN,NaN,249221109,201907,CONTINUA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,295641952,201910,1,0,1,61,251,0.00,0.00,0.00,...,1.0,0.001411,0.012774,NaN,NaN,NaN,NaN,295641952,201910,CONTINUA
499996,295641952,201911,1,0,1,61,252,6841.44,61155.08,2439.95,...,1.0,0.000452,0.019676,NaN,NaN,NaN,NaN,295641952,201911,CONTINUA
499997,295641952,201912,1,0,1,61,253,1284.34,58388.02,1823.62,...,1.0,0.001284,0.018416,NaN,NaN,NaN,NaN,295641952,201912,CONTINUA
499998,295641952,202001,1,0,1,61,254,11683.42,58697.96,13874.51,...,1.0,0.000876,0.018416,NaN,NaN,NaN,NaN,295641952,202001,CONTINUA


In [29]:
data['check'] = data.numero_de_cliente == data.numero_de_cliente_1

In [31]:
data.columns

Index(['numero_de_cliente', 'foto_mes', 'active_quarter', 'cliente_vip',
       'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad',
       'mrentabilidad_annual', 'mcomisiones',
       ...
       'tc_proporcion_consumo_dolares',
       'tc_proporcion_consumo_total_limite_total_cubierto',
       'tc_proporcion_pago_pesos', 'tc_proporcion_pago_dolares',
       'tc_proporcion_adelanto_pesos', 'tc_proporcion_adelanto_dolares',
       'numero_de_cliente_1', 'foto_mes_1', 'clase_ternaria_1', 'check'],
      dtype='object', length=239)

In [5]:
storage_name = "mysql+mysqldb://{u}:{p}@{ip}:3306/optuna_rf_db".format(p=urllib.parse.quote_plus(os.getenv("password")), u = os.getenv("usersrv"), ip = os.getenv("ip"))


In [13]:
st = optuna.load_study(study_name='experimento_sampling_10perc', storage=storage_name)

In [7]:
st.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_drop_rate,params_feature_fraction,params_feature_fraction_bynode,params_learning_rate,params_min_data_in_leaf,params_num_leaves,user_attrs_best_iter,user_attrs_max_bin,user_attrs_seed,user_attrs_train_months,state
0,0,NaN,2024-10-21 22:31:20,NaT,NaT,NaN,0.829601,NaN,0.158880,2715,4558,NaN,NaN,NaN,NaN,RUNNING
1,1,NaN,2024-10-22 00:48:10,2024-10-22 00:49:00,0 days 00:00:50,0.279233,0.742754,0.119214,0.285673,253,7680,NaN,NaN,NaN,NaN,FAIL
2,2,1.365443e+08,2024-10-22 00:51:16,2024-10-22 01:22:57,0 days 00:31:41,0.174322,0.291615,0.878297,0.127688,3184,2419,1933.0,NaN,NaN,"[202012, 202101, 202102, 202103, 202104, 202105]",COMPLETE
3,3,1.093750e+08,2024-10-22 01:23:05,2024-10-22 01:28:24,0 days 00:05:19,0.149678,0.336071,0.712341,0.332264,578,7538,71.0,NaN,NaN,"[202012, 202101, 202102, 202103, 202104, 202105]",COMPLETE
4,4,NaN,2024-10-22 01:28:31,NaT,NaT,0.029742,0.293263,0.219113,0.048427,1040,9094,NaN,NaN,NaN,NaN,RUNNING
5,5,NaN,2024-10-22 11:32:54,NaT,NaT,0.190402,0.056361,0.811841,0.251603,1170,2563,NaN,NaN,NaN,NaN,RUNNING
6,6,1.381298e+08,2024-10-22 11:59:41,2024-10-22 12:11:44,0 days 00:12:03,0.100160,0.132815,0.835478,0.137865,502,473,867.0,NaN,NaN,"[202012, 202101, 202102, 202103, 202104, 202105]",COMPLETE
7,7,1.263967e+08,2024-10-22 12:11:52,2024-10-22 12:28:06,0 days 00:16:14,0.074132,0.222099,0.294263,0.302919,2245,6690,950.0,NaN,NaN,"[202012, 202101, 202102, 202103, 202104, 202105]",COMPLETE
8,8,1.259580e+08,2024-10-22 12:28:13,2024-10-22 12:43:30,0 days 00:15:17,0.170795,0.258035,0.834443,0.328293,2907,7334,845.0,NaN,NaN,"[202012, 202101, 202102, 202103, 202104, 202105]",COMPLETE
9,9,1.250188e+08,2024-10-22 12:43:40,2024-10-22 12:58:25,0 days 00:14:45,0.109610,0.215280,0.555538,0.376307,3864,3818,1048.0,NaN,NaN,"[202012, 202101, 202102, 202103, 202104, 202105]",COMPLETE


In [8]:
st.best_trial()

FrozenTrial(number=38, state=1, values=[144669000.0], datetime_start=datetime.datetime(2024, 10, 23, 18, 14, 19), datetime_complete=datetime.datetime(2024, 10, 23, 19, 53, 52), params={'num_leaves': 974, 'learning_rate': 0.029680094869433238, 'min_data_in_leaf': 271, 'feature_fraction': 0.5511983327578917, 'feature_fraction_bynode': 0.8296806258635299, 'drop_rate': 0.2996545458819664}, user_attrs={'best_iter': 2468, 'max_bin': 100, 'seed': 549937, 'train_months': [202012, 202101, 202102, 202103, 202104, 202105]}, system_attrs={}, intermediate_values={}, distributions={'num_leaves': IntDistribution(high=10000, log=False, low=20, step=1), 'learning_rate': FloatDistribution(high=0.4, log=False, low=0.005, step=None), 'min_data_in_leaf': IntDistribution(high=4000, log=False, low=1, step=1), 'feature_fraction': FloatDistribution(high=0.9, log=False, low=0.005, step=None), 'feature_fraction_bynode': FloatDistribution(high=0.9, log=False, low=0.05, step=None), 'drop_rate': FloatDistribution(h

In [12]:
optuna.importance.get_param_importances(st)

{'learning_rate': 0.43203325382431934,
 'min_data_in_leaf': 0.2523585975269244,
 'feature_fraction': 0.12242100257306737,
 'drop_rate': 0.1191064771937908,
 'feature_fraction_bynode': 0.03830479637663614,
 'num_leaves': 0.03577587250526182}